In [1]:
%run Weighting.ipynb

Pixiedust database opened successfully


In [2]:
tabel_alternatif = open_tabel("Tabel Alternatif.csv")

tabel_alternatif

,Astronomi - ITB (IPA),Fakultas Ilmu Dan Teknologi Kebumian - ITB (IPA),Fakultas Teknik Sipil & Lingkungan - ITB (IPA),Fakultas Teknik Sipil Dan Lingkungan - ITB (IPA),Fakultas Teknik Sipil Dan Lingkungan (Ftsl) - ITB (IPA),Fakultas Teknik Sipil Dan Lingkungan-Kampus Jatinangor - ITB (IPA),Fakultas Teknologi Industri - ITB (IPA),Farmasi - ITB (IPA),Fitb - ITB (IPA),Fmipa - ITB (IPA),...,Sith S - ITB (IPA),Sith-R - ITB (IPA),Sith-S - ITB (IPA),Sithr - ITB (IPA),Stei - ITB (IPA),Teknik Industri - ITB (IPA),Teknik Kimia - ITB (IPA),Teknik Sipil - ITB (IPA),Teknik Sipil Dan Lingkungan - ITB (IPA),Teknik Sipil Lingkungan - ITB (IPA)
Matematika,86.000000,84.000000,89.675336,87.891779,87.000000,89.675336,90.337668,95.000000,88.284719,92.245022,...,93.000000,86.602601,93.000000,86.837668,94.000000,88.000000,80.000000,83.837668,85.000000,91.000000
Inggris,87.000000,86.000000,88.522222,90.000000,91.000000,88.522222,93.261111,90.000000,88.662393,88.634815,...,85.000000,88.356667,82.000000,85.761111,88.507407,90.000000,81.000000,83.261111,81.000000,87.000000
Indonesia,84.000000,85.000000,87.367816,91.666667,96.000000,87.367816,87.183908,84.000000,86.449160,87.715709,...,83.000000,87.210345,93.000000,85.183908,90.455939,87.000000,84.000000,82.183908,85.000000,84.000000
Fisika,92.000000,85.000000,88.747126,89.666667,94.000000,88.747126,88.873563,95.000000,87.883289,89.316475,...,88.000000,86.124138,91.000000,86.873563,88.582375,84.000000,82.000000,83.373563,86.000000,87.000000
Kimia,81.000000,83.000000,89.188218,88.666667,95.237644,89.188218,90.094109,98.000000,88.303161,90.279215,...,92.000000,84.956466,92.000000,85.594109,92.062739,86.000000,80.000000,83.594109,88.000000,92.000000
Biologi,85.000000,88.000000,88.355682,87.666667,88.000000,88.355682,91.177841,95.000000,86.775262,88.623712,...,93.000000,86.906705,96.000000,86.177841,88.785227,85.000000,85.000000,90.677841,92.000000,84.000000
Ekonomi,87.917424,87.917424,87.917424,87.917424,87.917424,87.917424,87.917424,87.917424,87.850664,88.198404,...,87.917424,87.828985,91.000000,87.917424,87.917424,87.917424,88.000000,87.917424,87.917424,87.917424
Geografi,85.858929,85.858929,85.858929,85.858929,85.858929,85.858929,85.858929,85.858929,86.474121,85.696714,...,85.858929,85.844571,85.858929,85.858929,85.858929,85.858929,85.858929,85.858929,85.858929,85.858929
Sosiologi,89.898990,89.898990,89.898990,89.898990,89.898990,89.898990,89.898990,89.898990,89.840559,89.163098,...,89.898990,89.898990,89.898990,89.898990,89.898990,89.898990,89.898990,89.898990,89.898990,89.898990


In [3]:
nilai_jams = [70, 70, 70, 70, 70, 70, 70, 70, 70,]

In [4]:
mean = get_mean(tabel_alternatif)
std = get_std(tabel_alternatif)
params = get_params(mean, nilai_jams, std)

params

[-5.2710142975652365,
 -6.243111836940413,
 -5.822917299193358,
 -5.770224587927644,
 -4.742990088112817,
 -6.171608357209187,
 -12.322015479719445,
 -23.124922712468912,
 -34.9354019427263]

In [5]:
tabel_alternatif = add_parameter(params, tabel_alternatif)

tabel_alternatif

,Astronomi - ITB (IPA),Fakultas Ilmu Dan Teknologi Kebumian - ITB (IPA),Fakultas Teknik Sipil & Lingkungan - ITB (IPA),Fakultas Teknik Sipil Dan Lingkungan - ITB (IPA),Fakultas Teknik Sipil Dan Lingkungan (Ftsl) - ITB (IPA),Fakultas Teknik Sipil Dan Lingkungan-Kampus Jatinangor - ITB (IPA),Fakultas Teknologi Industri - ITB (IPA),Farmasi - ITB (IPA),Fitb - ITB (IPA),Fmipa - ITB (IPA),...,Sith-R - ITB (IPA),Sith-S - ITB (IPA),Sithr - ITB (IPA),Stei - ITB (IPA),Teknik Industri - ITB (IPA),Teknik Kimia - ITB (IPA),Teknik Sipil - ITB (IPA),Teknik Sipil Dan Lingkungan - ITB (IPA),Teknik Sipil Lingkungan - ITB (IPA),Parameter
Matematika,86.000000,84.000000,89.675336,87.891779,87.000000,89.675336,90.337668,95.000000,88.284719,92.245022,...,86.602601,93.000000,86.837668,94.000000,88.000000,80.000000,83.837668,85.000000,91.000000,-5.271014
Inggris,87.000000,86.000000,88.522222,90.000000,91.000000,88.522222,93.261111,90.000000,88.662393,88.634815,...,88.356667,82.000000,85.761111,88.507407,90.000000,81.000000,83.261111,81.000000,87.000000,-6.243112
Indonesia,84.000000,85.000000,87.367816,91.666667,96.000000,87.367816,87.183908,84.000000,86.449160,87.715709,...,87.210345,93.000000,85.183908,90.455939,87.000000,84.000000,82.183908,85.000000,84.000000,-5.822917
Fisika,92.000000,85.000000,88.747126,89.666667,94.000000,88.747126,88.873563,95.000000,87.883289,89.316475,...,86.124138,91.000000,86.873563,88.582375,84.000000,82.000000,83.373563,86.000000,87.000000,-5.770225
Kimia,81.000000,83.000000,89.188218,88.666667,95.237644,89.188218,90.094109,98.000000,88.303161,90.279215,...,84.956466,92.000000,85.594109,92.062739,86.000000,80.000000,83.594109,88.000000,92.000000,-4.742990
Biologi,85.000000,88.000000,88.355682,87.666667,88.000000,88.355682,91.177841,95.000000,86.775262,88.623712,...,86.906705,96.000000,86.177841,88.785227,85.000000,85.000000,90.677841,92.000000,84.000000,-6.171608
Ekonomi,87.917424,87.917424,87.917424,87.917424,87.917424,87.917424,87.917424,87.917424,87.850664,88.198404,...,87.828985,91.000000,87.917424,87.917424,87.917424,88.000000,87.917424,87.917424,87.917424,-12.322015
Geografi,85.858929,85.858929,85.858929,85.858929,85.858929,85.858929,85.858929,85.858929,86.474121,85.696714,...,85.844571,85.858929,85.858929,85.858929,85.858929,85.858929,85.858929,85.858929,85.858929,-23.124923
Sosiologi,89.898990,89.898990,89.898990,89.898990,89.898990,89.898990,89.898990,89.898990,89.840559,89.163098,...,89.898990,89.898990,89.898990,89.898990,89.898990,89.898990,89.898990,89.898990,89.898990,-34.935402


In [6]:
pilihan_jurusan = get_jurusan(tabel_alternatif)

for pilihan in pilihan_jurusan:
    if "(ipa)" in pilihan.casefold().split():
        print(pilihan)

Astronomi - ITB (IPA)
Fakultas Ilmu Dan Teknologi Kebumian - ITB (IPA)
Fakultas Teknik Sipil & Lingkungan - ITB (IPA)
Fakultas Teknik Sipil Dan Lingkungan - ITB (IPA)
Fakultas Teknik Sipil Dan Lingkungan (Ftsl) - ITB (IPA)
Fakultas Teknik Sipil Dan Lingkungan-Kampus Jatinangor - ITB (IPA)
Fakultas Teknologi Industri - ITB (IPA)
Farmasi - ITB (IPA)
Fitb - ITB (IPA)
Fmipa - ITB (IPA)
Fti - ITB (IPA)
Fti-G - ITB (IPA)
Fti-Ganesa - ITB (IPA)
Ftmd - ITB (IPA)
Ftsl - ITB (IPA)
Ftsl Ganesa - ITB (IPA)
Ftsl Ganesha - ITB (IPA)
Ftsl Kampus Ganesa - ITB (IPA)
Ftsl-G - ITB (IPA)
Ftsl-Ganesha - ITB (IPA)
Ftsl-J - ITB (IPA)
Fttm  - ITB (IPA)
Itb - Sithr - ITB (IPA)
Matematika Aktuaria - ITB (IPA)
Sappk - ITB (IPA)
School Of Business And Management  - ITB (IPA)
Sekolah Farmasi - ITB (IPA)
Sekolah Ilmu Dan Tekno Hayati - Program Rekayasa - ITB (IPA)
Sekolah Ilmu Dan Teknologi Hayati - Sains - ITB (IPA)
Sith - Rekayasa - ITB (IPA)
Sith S - ITB (IPA)
Sith-R - ITB (IPA)
Sith-S - ITB (IPA)
Sithr - ITB (I

In [7]:
fp_gaussian = fungsi_preferensi_gaussian(tabel_alternatif)

fp_gaussian

,Astronomi - ITB (IPA)&Fakultas Ilmu Dan Teknologi Kebumian - ITB (IPA),Astronomi - ITB (IPA)&Fakultas Teknik Sipil & Lingkungan - ITB (IPA),Astronomi - ITB (IPA)&Fakultas Teknik Sipil Dan Lingkungan - ITB (IPA),Astronomi - ITB (IPA)&Fakultas Teknik Sipil Dan Lingkungan (Ftsl) - ITB (IPA),Astronomi - ITB (IPA)&Fakultas Teknik Sipil Dan Lingkungan-Kampus Jatinangor - ITB (IPA),Astronomi - ITB (IPA)&Fakultas Teknologi Industri - ITB (IPA),Astronomi - ITB (IPA)&Farmasi - ITB (IPA),Astronomi - ITB (IPA)&Fitb - ITB (IPA),Astronomi - ITB (IPA)&Fmipa - ITB (IPA),Astronomi - ITB (IPA)&Fti - ITB (IPA),...,Teknik Sipil Lingkungan - ITB (IPA)&Sith - Rekayasa - ITB (IPA),Teknik Sipil Lingkungan - ITB (IPA)&Sith S - ITB (IPA),Teknik Sipil Lingkungan - ITB (IPA)&Sith-R - ITB (IPA),Teknik Sipil Lingkungan - ITB (IPA)&Sith-S - ITB (IPA),Teknik Sipil Lingkungan - ITB (IPA)&Sithr - ITB (IPA),Teknik Sipil Lingkungan - ITB (IPA)&Stei - ITB (IPA),Teknik Sipil Lingkungan - ITB (IPA)&Teknik Industri - ITB (IPA),Teknik Sipil Lingkungan - ITB (IPA)&Teknik Kimia - ITB (IPA),Teknik Sipil Lingkungan - ITB (IPA)&Teknik Sipil - ITB (IPA),Teknik Sipil Lingkungan - ITB (IPA)&Teknik Sipil Dan Lingkungan - ITB (IPA)
Matematika,0.0,0.5,0.0,0.0,0.5,0.5,1.0,0.5,1.0,0.5,...,1,0,0.5,0.0,0.5,0.5,0.5,1.0,1.0,1.0
Inggris,0.0,0.0,0.5,0.5,0.0,1.0,0.5,0.0,0.0,0.5,...,0,0,0.0,0.5,0.0,0.0,0.5,1.0,0.5,1.0
Indonesia,0.0,0.5,1.0,1.0,0.5,0.5,0.0,0.5,0.5,1.0,...,0,0,0.5,1.0,0.0,1.0,0.5,0.0,0.0,0.0
Fisika,1.0,0.5,0.5,0.0,0.5,0.5,0.5,0.5,0.5,0.0,...,0,0,0.0,0.5,0.0,0.0,0.5,0.5,0.5,0.0
Kimia,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1,0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.5
Biologi,0.5,0.5,0.5,0.5,0.5,1.0,1.0,0.0,0.5,0.5,...,0,1,0.5,1.0,0.5,0.5,0.0,0.0,1.0,1.0
Ekonomi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Geografi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Sosiologi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
index_preferensi_global = idx_pref_global(fp_gaussian, weight)

index_preferensi_global

,Astronomi - ITB (IPA)&Fakultas Ilmu Dan Teknologi Kebumian - ITB (IPA),Astronomi - ITB (IPA)&Fakultas Teknik Sipil & Lingkungan - ITB (IPA),Astronomi - ITB (IPA)&Fakultas Teknik Sipil Dan Lingkungan - ITB (IPA),Astronomi - ITB (IPA)&Fakultas Teknik Sipil Dan Lingkungan (Ftsl) - ITB (IPA),Astronomi - ITB (IPA)&Fakultas Teknik Sipil Dan Lingkungan-Kampus Jatinangor - ITB (IPA),Astronomi - ITB (IPA)&Fakultas Teknologi Industri - ITB (IPA),Astronomi - ITB (IPA)&Farmasi - ITB (IPA),Astronomi - ITB (IPA)&Fitb - ITB (IPA),Astronomi - ITB (IPA)&Fmipa - ITB (IPA),Astronomi - ITB (IPA)&Fti - ITB (IPA),...,Teknik Sipil Lingkungan - ITB (IPA)&Sith - Rekayasa - ITB (IPA),Teknik Sipil Lingkungan - ITB (IPA)&Sith S - ITB (IPA),Teknik Sipil Lingkungan - ITB (IPA)&Sith-R - ITB (IPA),Teknik Sipil Lingkungan - ITB (IPA)&Sith-S - ITB (IPA),Teknik Sipil Lingkungan - ITB (IPA)&Sithr - ITB (IPA),Teknik Sipil Lingkungan - ITB (IPA)&Stei - ITB (IPA),Teknik Sipil Lingkungan - ITB (IPA)&Teknik Industri - ITB (IPA),Teknik Sipil Lingkungan - ITB (IPA)&Teknik Kimia - ITB (IPA),Teknik Sipil Lingkungan - ITB (IPA)&Teknik Sipil - ITB (IPA),Teknik Sipil Lingkungan - ITB (IPA)&Teknik Sipil Dan Lingkungan - ITB (IPA)
Index Preferensi Global,0.155836,0.344164,0.363616,0.311671,0.344164,0.5,0.480548,0.292219,0.428603,0.39611,...,0.272767,0.10389,0.292219,0.311671,0.240274,0.240274,0.344164,0.428603,0.480548,0.428603


In [9]:
e_flow = entering_flow(pilihan_jurusan, index_preferensi_global)

e_flow

{'Astronomi - ITB (IPA)': 0.34194860088023155,
 'Fakultas Ilmu Dan Teknologi Kebumian - ITB (IPA)': 0.3487685663979211,
 'Fakultas Teknik Sipil & Lingkungan - ITB (IPA)': 0,
 'Fakultas Teknik Sipil Dan Lingkungan - ITB (IPA)': 0.27749020623151344,
 'Fakultas Teknik Sipil Dan Lingkungan (Ftsl) - ITB (IPA)': 0.38138058628582616,
 'Fakultas Teknik Sipil Dan Lingkungan-Kampus Jatinangor - ITB (IPA)': 0.2310819013865821,
 'Fakultas Teknologi Industri - ITB (IPA)': 0.28017509703475457,
 'Farmasi - ITB (IPA)': 0.40973342649992284,
 'Fitb - ITB (IPA)': 0.24739597357110646,
 'Fmipa - ITB (IPA)': 0.22965859161061405,
 'Fti - ITB (IPA)': 0.24470033311376965,
 'Fti-G - ITB (IPA)': 0.22458539336628275,
 'Fti-Ganesa - ITB (IPA)': 0.3541329731773554,
 'Ftmd - ITB (IPA)': 0.23709859798784433,
 'Ftsl - ITB (IPA)': 0.21809426017303107,
 'Ftsl Ganesa - ITB (IPA)': 0.26798534991058287,
 'Ftsl Ganesha - ITB (IPA)': 0.22664486848293508,
 'Ftsl Kampus Ganesa - ITB (IPA)': 0.26814707865927784,
 'Ftsl-G - ITB 

In [10]:
l_flow = leaving_flow(pilihan_jurusan, index_preferensi_global)

l_flow

{'Astronomi - ITB (IPA)': 0.3653899715986919,
 'Fakultas Ilmu Dan Teknologi Kebumian - ITB (IPA)': 0.3763288873496863,
 'Fakultas Teknik Sipil & Lingkungan - ITB (IPA)': 0,
 'Fakultas Teknik Sipil Dan Lingkungan - ITB (IPA)': 0.28208359305680764,
 'Fakultas Teknik Sipil Dan Lingkungan (Ftsl) - ITB (IPA)': 0.4040294377419549,
 'Fakultas Teknik Sipil Dan Lingkungan-Kampus Jatinangor - ITB (IPA)': 0.23440833235755545,
 'Fakultas Teknologi Industri - ITB (IPA)': 0.286509876306359,
 'Farmasi - ITB (IPA)': 0.44283600746096097,
 'Fitb - ITB (IPA)': 0.2519893603964007,
 'Fmipa - ITB (IPA)': 0.2350444976982398,
 'Fti - ITB (IPA)': 0.24850120067673237,
 'Fti-G - ITB (IPA)': 0.22331843751196187,
 'Fti-Ganesa - ITB (IPA)': 0.37598930537115255,
 'Ftmd - ITB (IPA)': 0.2421664214051279,
 'Ftsl - ITB (IPA)': 0.21936121602735198,
 'Ftsl Ganesa - ITB (IPA)': 0.2784390794154922,
 'Ftsl Ganesha - ITB (IPA)': 0.2329796477545395,
 'Ftsl Kampus Ganesa - ITB (IPA)': 0.27939332742651873,
 'Ftsl-G - ITB (IPA)':

In [11]:
n_flow = net_flow(pilihan_jurusan, e_flow, l_flow)

n_flow

{'Astronomi - ITB (IPA)': 0.023441370718460364,
 'Fakultas Ilmu Dan Teknologi Kebumian - ITB (IPA)': 0.027560320951765194,
 'Fakultas Teknik Sipil & Lingkungan - ITB (IPA)': 0,
 'Fakultas Teknik Sipil Dan Lingkungan - ITB (IPA)': 0.004593386825294199,
 'Fakultas Teknik Sipil Dan Lingkungan (Ftsl) - ITB (IPA)': 0.022648851456128716,
 'Fakultas Teknik Sipil Dan Lingkungan-Kampus Jatinangor - ITB (IPA)': 0.0033264309709733486,
 'Fakultas Teknologi Industri - ITB (IPA)': 0.006334779271604418,
 'Farmasi - ITB (IPA)': 0.03310258096103813,
 'Fitb - ITB (IPA)': 0.004593386825294227,
 'Fmipa - ITB (IPA)': 0.005385906087625736,
 'Fti - ITB (IPA)': 0.0038008675629627175,
 'Fti-G - ITB (IPA)': -0.001266955854320878,
 'Fti-Ganesa - ITB (IPA)': 0.021856332193797123,
 'Ftmd - ITB (IPA)': 0.005067823417283568,
 'Ftsl - ITB (IPA)': 0.0012669558543209058,
 'Ftsl Ganesa - ITB (IPA)': 0.010453729504909304,
 'Ftsl Ganesha - ITB (IPA)': 0.006334779271604418,
 'Ftsl Kampus Ganesa - ITB (IPA)': 0.011246248767

In [12]:
hasil_outrank = sorted(n_flow.items(), key = lambda kv:(kv[1], kv[0]), reverse = True)

tab = dict()

rank = list()
jurusan = list()
net = list()

for i in range(len(hasil_outrank)):
    rank.append(i+1)
    jurusan.append(hasil_outrank[i][0])
    net.append(hasil_outrank[i][1])
    
tab["Jurusan"] = jurusan
tab["Net Flow"] = net

tabel_hasil = pd.DataFrame(tab, index=rank)

tabel_hasil

,Jurusan,Net Flow
1,Ftsl-Ganesha - ITB (IPA),0.036429
2,Teknik Kimia - ITB (IPA),0.035162
3,Teknik Sipil - ITB (IPA),0.035162
4,Farmasi - ITB (IPA),0.033103
5,Itb - Sithr - ITB (IPA),0.027560
6,Fakultas Ilmu Dan Teknologi Kebumian - ITB (IPA),0.027560
7,Sith - Rekayasa - ITB (IPA),0.025026
8,Astronomi - ITB (IPA),0.023441
9,Fakultas Teknik Sipil Dan Lingkungan (Ftsl) - ...,0.022649
10,Fti-Ganesa - ITB (IPA),0.021856
